# Heatmap in Plotly
 experimental interactive plots
 TODO: pretty much everything

In [26]:
import sys
import os

import numpy as np
import pandas as pd
import plotly.express as px

sys.path.insert(0, os.path.abspath('../..'))
from src.util.db_utils import SynFermDatabaseConnection
from src.util.pandas_utils import alphanumeric_index_sort
from src.util.utils import get_conf
from src.definitions import DATA_DIR
con = SynFermDatabaseConnection()
conf = get_conf()

In [28]:
df = pd.read_csv(DATA_DIR / "curated_data" / "synferm_dataset_2023-07-20_40433records.csv")
len(df)

40433

In [29]:
df.columns

Index(['experiment_id', 'I_long', 'M_long', 'T_long', 'product_A_smiles',
       'I_smiles', 'M_smiles', 'T_smiles', 'reaction_smiles',
       'reaction_smiles_atom_mapped', 'binary_A', 'binary_B', 'binary_C',
       'binary_D', 'binary_E', 'binary_F', 'binary_G', 'binary_H', 'scaled_A',
       'scaled_B', 'scaled_C', 'scaled_D', 'scaled_E', 'scaled_F', 'scaled_G',
       'scaled_H', 'major_A-C'],
      dtype='object')

In [32]:
# prepare data for plotting
y = "M_long"
x = "I_long"
product = "A"
df_heatmap1 = df.groupby([y, x]).mean(numeric_only=True)[f"binary_{product}"].unstack()
df_heatmap1 = df_heatmap1.sort_index()
df_heatmap1 = df_heatmap1.sort_index(axis=1)

In [36]:
fig = px.imshow(df_heatmap1, color_continuous_scale="Viridis", width=800, height=800)
fig.show()

In [39]:


# Calculate the average target values for the selected variables
df_heatmap = df.groupby(['I_long', 'M_long'])['binary_A'].mean().reset_index()

# Create the initial heatmap with the average target values
fig = px.imshow(df_heatmap)

# Define the update function
def update_heatmap(selected_variable1, selected_variable2):
    # Calculate the average target values for the selected variables
    df_heatmap = df.groupby([selected_variable1, selected_variable2])['binary_A'].mean().reset_index()
    
    # Update the data shown in the heatmap based on the user's selection
    fig.data[0].x = df_heatmap[selected_variable2]
    fig.data[0].y = df_heatmap[selected_variable1]
    fig.data[0].z = df_heatmap['binary_A']

# Add the dropdown menus
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=[{'x': [df_heatmap['M_long']], 'y': [df_heatmap['I_long']], 'z': [df_heatmap['binary_A']]}],
                    label='Variable 1 and Variable 2',
                    method='restyle'
                ),
                dict(
                    args=[{'x': [df_heatmap['T_long']], 'y': [df_heatmap['M_long']], 'z': [df_heatmap['binary_A']]}],
                    label='Variable 2 and Variable 3',
                    method='restyle'
                )
                
            ]),
            direction='down',
            showactive=True,
            x=0.1,
            xanchor='left',
            y=1.1,
            yanchor='top'
        ),
    ]
)

# Display the plot
fig.show()


KeyError: 'T_long'

In [37]:
fig.write_html('/Users/julian/WebProjects/synferm-plot/test.html', include_plotlyjs='cdn', config={"displayModeBar": True})